# Double integrating sphere measurements

**Scott Prahl**

**Dec 2021**

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import iadpython.sphere
%config InlineBackend.figure_format='retina'

## Double Integrating Spheres

One day, this will be filled with two sphere experiment information.

## Port areas

If we look at a cross-section of the double integrating sphere setup used to measure reflection and transmission simultaneously then we have

<center>
    <img src="two-sphere-basic.png" width="80%" />
</center>

The spheres can be different sizes with different wall reflectivities, but port size for the sample on both spheres must be the same.

The odd thing about the double integrating sphere set up is that light that has passed through the sample bounces around and can hit the sample from (what was) the backside (shown in red).


However, the power of light falling on the detector differs from single sphere measurements because light may repeatedly pass back and forth through the sample between the two spheres. Thus, the gain becomes more complicated and now depends on both the reflectance and transmittance of the sample.  In the following derivation, the subscript 1 refers to the reflection sphere and the subscript 2 to the transmission sphere and arrows are used to denote the direction of the source to the destination sphere.

## Reflection sphere

**Have to worry about specular and light hitting wall first!**

We will track the light in the reflectance sphere by the number of times it passes through the sample.  The light falling on the detector port in the reflection sphere is
$$
P^r = a_d^r \sum_{k=0}^{\infty} P^x_{2k}
$$
where $P^x_k$ is the power in the sphere after $k$ passes through the sample.  The even numbers of passes end up in the reflectance sphere and the
odd numbers of passes end up in the transmission sphere.

Light that does not pass through the sample at all can be treated just like the light from a single sphere.  The power in the reflectance sphere for an incident power $P_i$ will be
$$
P_0 = G^r \cdot \mathrm{UR1} \cdot P_i
$$
where $G_r$ is the gain due to the reflectance sphere.

The power in the transmission sphere for light making a single pass through the sample once will be a combination of the transmitted light for direct illumination and the diffuse light from the first bounce
$$
P_1 = G^t \left[ \mathrm{UT1} \cdot P_i + \mathrm{UTU} \cdot a_s P_0 \right] = \left[ G^t \mathrm{UT1} + a_s G^r G^t \cdot \mathrm{UR1} \cdot\mathrm{UTU} \right]  P_i
$$
The power in the reflectance sphere for light that has traversed the sample twice is
$$
P_2 = G^r \cdot \mathrm{UTU} \cdot a_s \cdot P_1
$$
this can be generalized to
$$
P_{2k} =  G^r \cdot \mathrm{UTU} \cdot a_s\cdot P_{2k-1}
\qquad\qquad k\ge1
$$

The light in the transmission sphere for light thas has made $2k-1$ passes through the sample is
$$
P_{2k-1} =  G^t \cdot \mathrm{UTU} \cdot a_s\cdot P_{2k-2}
\qquad\qquad k>1
$$
The power in the reflectance sphere
$$
P_{2k} =  a_s^2 G^r G^t \mathrm{UTU}^2 \cdot P_{2k-2}
\qquad\qquad k>1
$$
or more generally
$$
P_{2k} =  (a_s^2 G^r G^t  \mathrm{UTU}^2)^{k-1} \cdot P_2
\qquad\qquad k>0
$$

If $\alpha=G^r G^t a_s^2 \mathrm{UTU}^2$, then
$$
P^r = P_0 + P_2 + P_4 + \cdots = P_0 + P_2\left[1 + \alpha + \alpha^2 +\cdots \right]
$$
and summing we get
$$
P^r = P_0 + \frac{P_2}{1-\alpha}
$$

To see how the UR1 and UT1 affect this value, we expand and do some algebra
$$
\frac{P^r}{P_i} =  G^r \cdot \mathrm{UR1} + G^r \cdot \mathrm{UTU} \cdot a_s \frac{G^t\mathrm{UT1} + a_s G^r G^t\cdot \mathrm{UR1} \cdot\mathrm{UTU}}{1-\alpha}
$$
or
$$
\frac{P^r}{P_i} =  G^r\cdot\mathrm{UR1}\left[ 1  + \frac{a_s^2  G^r G^t \cdot\mathrm{UTU}^2}{1-\alpha}\right]
+G^r\cdot \mathrm{UT1} \left[\frac{a_s G^t \cdot\mathrm{UTU}}{1-\alpha}\right]
$$
or
$$
\frac{P^r}{P_i} = \mathrm{UR1} \cdot G^r\left[ \frac{1}{1-\alpha}\right]
+ \mathrm{UT1} \cdot G^r\left[\frac{a_s  G^t \cdot\mathrm{UTU}}{1-\alpha} \right]
$$
or
$$
\frac{P^r}{P_i} = \left[ \frac{G^r}{1-\alpha} \right]
\bigg( \mathrm{UR1}  +  \mathrm{UT1} \cdot a_s  G^t \,\mathrm{UTU} \bigg)
$$



In [152]:
N = 10
P = np.zeros(N)
Psum = np.zeros(N)
Podd = np.zeros(N)

ur1 = 0.3
uru = 0.5
ut1 = 0.5
utu = 0.2

r_sphere = iadpython.sphere.Sphere(100, 40, r_wall=0.98, d_third=10)
t_sphere = iadpython.sphere.Sphere(100, 40, r_wall=0.97, refl=False)

Gr = r_sphere.gain(sample_uru=uru)
Gt = t_sphere.gain(sample_uru=uru)
a_sample = r_sphere.sample.a
alpha = a_sample**2 * Gr * Gt * utu**2

print("a_s     = %7.3f" % (a_sample))
print("G_r     = %7.3f" % Gr)
print("G_t     = %7.3f" % Gt)
print("G_r*a_s = %7.3f" % (Gr*a_sample))
print('⍺       = %10.7f'% alpha)
print('1/(1-⍺) = %10.7f'% (1/(1-alpha)))
print()

P[0] = Gr * ur1
P[1] = Gt * (ut1 + utu * a_sample * P[0])

for k in range(1,N//2):
    P[2*k] = P[2*k-1] * Gr * a_sample * utu
    P[2*k+1] = P[2*k] * Gt * a_sample * utu

Psum[0] = P[0]
for k in range(1,N//2):
    Psum[2*k] = P[2*k] + Psum[2*k-2]

Podd[1] = P[1]
for k in range(1,N//2):
    Podd[2*k+1] = P[2*k+1] + Podd[2*k-1]

Pr = Gr /(1-alpha) * (ur1 + ut1*Gt*a_sample*utu)
Pr = P[0] + P[2]/(1-alpha)
Pt = P[1] + Gt*a_sample*utu * P[2]/(1-alpha)
Pt = P[1] + alpha * P[1]/(1-alpha)
Pt = P[1]/(1-alpha)

print('    k    P[k]       P_even  P_r        P_odd   P_t')
for k in range(10):
    print("%5d %7.4f  -- %7.4f %7.4f -- %7.4f %7.4f" % (k, P[k], Psum[k], Pr, Podd[k], Pt))

a_s     =   0.040
G_r     =  24.010
G_t     =  20.492
G_r*a_s =   0.960
⍺       =  0.0314880
1/(1-⍺) =  1.0325117

    k    P[k]       P_even  P_r        P_odd   P_t
    0  7.2029  --  7.2029  9.4690 --  0.0000 11.7982
    1 11.4267  --  0.0000  9.4690 -- 11.4267 11.7982
    2  2.1948  --  9.3977  9.4690 --  0.0000 11.7982
    3  0.3598  --  0.0000  9.4690 -- 11.7865 11.7982
    4  0.0691  --  9.4668  9.4690 --  0.0000 11.7982
    5  0.0113  --  0.0000  9.4690 -- 11.7978 11.7982
    6  0.0022  --  9.4690  9.4690 --  0.0000 11.7982
    7  0.0004  --  0.0000  9.4690 -- 11.7982 11.7982
    8  0.0001  --  9.4690  9.4690 --  0.0000 11.7982
    9  0.0000  --  0.0000  9.4690 -- 11.7982 11.7982


## Transmission sphere


The light in the transmission sphere that has made $2k+1$ passes through the sample is
$$
P_{2k+1} =  G^t \cdot \mathrm{UTU} \cdot a_s\cdot P_{2k}
\qquad\qquad k\ge1
$$
Since 
$$
P_{2k} =  \alpha^{k-1} \cdot P_2
\qquad\qquad k>0
$$
We can express the contribution for $2k+1$ passes in terms of the first pass
$$
P_{2k+1} = (a_s\cdot  G^t \cdot \mathrm{UTU}) \cdot  \alpha^{k-1} \cdot P_2 = (a_s\cdot  G^t \cdot \mathrm{UTU} )\cdot  \alpha^{k-1} \cdot (G^r \cdot \mathrm{UTU} \cdot a_s \cdot P_1)
\qquad\qquad k>0
$$
which can be simplified down to
$$
P_{2k+1} = (a_s^2\cdot  G^r G^t \cdot \mathrm{UTU}^2 ) \cdot \alpha^{k-1} \cdot P_1 = \alpha^k  P_1
\qquad\qquad k>0
$$

The total light in the transmission sphere is the sum of all the odd terms
$$
P^t = P_1 + P_3 + P_5 + \cdots = P_1 + P_1 \alpha \left[1 + \alpha + \alpha^2 +\cdots \right] = P_1 + P_1 \frac{\alpha}{1-\alpha}
$$
which can be simplified to
$$
P^t  = \frac{P_1}{1-\alpha}
$$



## Accounting for other sphere effects

There are three other possible issues.  First, a fraction of the incident light may hit the reflectance sphere wall first.  Second, not all of the unscattered reflected light might be collected.  Third, the unscattered transmitted light might leave the transmission sphere.

This affects the values of $P_0$ and $P_1$ 